In [1]:
%load_ext autoreload
%autoreload 2

# HyDE (Hypothetical Document Embeddings) in LLMs
- HyDE (Hypothetical Document Embeddings) is a retrieval-augmented generation (RAG) technique used to improve search and retrieval in LLM-powered applications. Instead of directly embedding a query and searching for similar documents, HyDE first generates a hypothetical answer using an LLM and then embeds that answer for retrieval.

# How HyDE Works
1. User Query → Hypothetical Answer Generation
- Given a query, an LLM generates a hypothetical response (even if it's not perfect).

2. Hypothetical Answer → Embedding Model
- The generated response is converted into an embedding (vector representation).

3. Vector Search → Retrieve Similar Real Documents
- The system searches the vector database using the embedding of the hypothetical answer instead of the original query.
- This often leads to better matches than using the raw query alone.

4. Final RAG Response
- The most relevant retrieved documents are fed back into the LLM to generate the final answer.


In [5]:
import os
from typing import Any, List, Literal

import streamlit as st
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.memory import ChatMessageHistory
from langchain.prompts import (
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain.schema.output_parser import StrOutputParser
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.language_models import BaseChatModel
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.retrievers import BaseRetriever
from langchain_core.runnables import RunnablePassthrough, chain
from langchain_core.runnables.base import RunnableSerializable
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings

from loguru import logger
from rich.pretty import pprint

In [6]:
def pretty_print(title: str = None, content: Any = None):
    if title is None:
        print(content)
        return
    print(title)
    pprint(content)


st.session_state["history"] = (
    ChatMessageHistory()
    if "history" not in st.session_state
    else st.session_state["history"]
)

2025-03-19 09:57:24.631 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-19 09:57:24.631 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-03-19 09:57:24.631 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-19 09:57:24.632 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
